In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import glob
import tqdm as tqdm
import random
import tensorflow as tf
from tensorflow import keras

In [3]:
column_names=["file_path","label"]
train=pd.DataFrame(columns=column_names)
test=pd.DataFrame(columns=column_names)

In [4]:
for df,folder in zip([train,test],['train','test']):
    file_paths=glob.glob("/kaggle/input/cifar10/cifar10/"+folder+"/*/*")
    df['file_path']=file_paths
    labels=[]
    for file_path in tqdm.tqdm(file_paths):
        label=file_path.split('_')[-1].split('.')[0]
        labels.append(label)
    df['label']=labels

100%|██████████| 10000/10000 [00:00<00:00, 498449.62it/s]


In [5]:
train

,file_path,label
0,/kaggle/input/cifar10/cifar10/train/airplane/2...,airplane
1,/kaggle/input/cifar10/cifar10/train/airplane/2...,airplane
2,/kaggle/input/cifar10/cifar10/train/airplane/4...,airplane
3,/kaggle/input/cifar10/cifar10/train/airplane/1...,airplane
4,/kaggle/input/cifar10/cifar10/train/airplane/4...,airplane
...,...,...
49995,/kaggle/input/cifar10/cifar10/train/deer/41608...,deer
49996,/kaggle/input/cifar10/cifar10/train/deer/6785_...,deer
49997,/kaggle/input/cifar10/cifar10/train/deer/17777...,deer
49998,/kaggle/input/cifar10/cifar10/train/deer/9202_...,deer


In [6]:
def final_data(path,label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[32,32])
    image = image/255
    return image,label

In [7]:
def transform(df,training=False):
    labels=pd.get_dummies(df.label)
    preliminary_data=list(zip(df['file_path'],labels.values.tolist()))
    if(training==True):
        random.shuffle(preliminary_data)
    paths,labels=zip(*preliminary_data)
    data=tf.data.Dataset.from_tensor_slices((list(paths),list(labels)))
    data=data.map(final_data)
    return data

In [8]:
train_data=transform(train,training=True)

In [9]:
test_data=transform(test)

In [10]:
test_data=test_data.take(10000)

In [11]:
val_size=round(0.1*50000)
train_size=round(0.9*50000)
train_data=train_data.take(train_size).prefetch(30)
val_data=train_data.skip(train_size)
val_data=train_data.take(val_size).prefetch(30)

In [12]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(10,), dtype=tf.int32, name=None))>

In [22]:
mirrored_strategy=tf.distribute.MirroredStrategy()

In [23]:
#model 5
# with mirrored_strategy.scope():
#     model=tf.keras.Sequential()
#     model.add(tf.keras.layers.Conv2D(30,(2,2),strides=(1,1),padding='valid',
#                                      activation='relu',input_shape=(32,32,3)))
#     model.add(tf.keras.layers.Conv2D(30,(2,2),strides=(1,1),padding='valid',activation='relu'))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=None,padding='valid'))
# #   model.add(tf.keras.layers.Conv2D(20,(2,2),strides=(1,1),padding='valid',activation='relu'))
#     model.add(tf.keras.layers.Conv2D(15,(3,3),strides=(1,1),padding='valid',activation='relu'))
# #     model.add(tf.keras.layers.Conv2D(15,(3,3),strides=(1,1),padding='valid',activation='relu'))
#     model.add(tf.keras.layers.GroupNormalization(groups=3))
#     model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=None,padding='valid'))
#     model.add(tf.keras.layers.Conv2D(10,(3,3),strides=(1,1),padding='valid',activation='relu'))
#     model.add(tf.keras.layers.Flatten())
#     model.add(tf.keras.layers.Normalization())
#     model.add(tf.keras.layers.Dense(128,activation='relu')) #reduce neurons?
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dropout(0.1))
#     model.add(tf.keras.layers.Dense(64,activation='relu'))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dropout(0.1))
#     model.add(tf.keras.layers.Dense(32,activation='relu'))
#     model.add(tf.keras.layers.Dense(10,activation='softmax'))
#     model.compile(optimizer="Adam",loss='categorical_crossentropy',metrics=['accuracy'])

In [24]:
#model 6
# with mirrored_strategy.scope():
#     model=tf.keras.Sequential()
#     model.add(tf.keras.layers.Conv2D(30,(2,2),strides=(1,1),padding='valid',
#                                      activation='relu',input_shape=(32,32,3)))
#     model.add(tf.keras.layers.Conv2D(30,(2,2),strides=(1,1),padding='valid',activation='relu'))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.AveragePooling2D(pool_size=(2,2),strides=None,padding='valid'))
# #   model.add(tf.keras.layers.Conv2D(20,(2,2),strides=(1,1),padding='valid',activation='relu'))
#     model.add(tf.keras.layers.Conv2D(15,(2,2),strides=(1,1),padding='valid',activation='relu'))
# #     model.add(tf.keras.layers.Conv2D(15,(3,3),strides=(1,1),padding='valid',activation='relu'))
#     model.add(tf.keras.layers.GroupNormalization(groups=3))
#     model.add(tf.keras.layers.AveragePooling2D(pool_size=(2,2),strides=None,padding='valid'))
#     model.add(tf.keras.layers.Conv2D(10,(2,2),strides=(1,1),padding='valid',activation='relu'))
#     model.add(tf.keras.layers.Flatten())
#     model.add(tf.keras.layers.Normalization())
#     model.add(tf.keras.layers.Dense(128,activation='relu')) #reduce neurons?
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dropout(0.3)) #0.1?
#     model.add(tf.keras.layers.Dense(64,activation='relu'))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dropout(0.3)) #0.1?
#     model.add(tf.keras.layers.Dense(32,activation='relu'))
#     model.add(tf.keras.layers.Dense(10,activation='softmax'))
#     model.compile(optimizer="Adam",loss='categorical_crossentropy',metrics=['accuracy'])

In [25]:
# with mirrored_strategy.scope():
#     model=tf.keras.Sequential()
#     model.add(tf.keras.layers.Conv2D(30,(3,3),strides=(1,1),padding='valid',
#                                      activation='relu',input_shape=(32,32,3)))
#     model.add(tf.keras.layers.Conv2D(30,(3,3),strides=(1,1),padding='valid',activation='relu'))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.AveragePooling2D(pool_size=(2,2),strides=None,padding='valid'))
# #   model.add(tf.keras.layers.Conv2D(20,(2,2),strides=(1,1),padding='valid',activation='relu'))
#     model.add(tf.keras.layers.Conv2D(15,(2,2),strides=(1,1),padding='valid',activation='relu'))
# #     model.add(tf.keras.layers.Conv2D(15,(3,3),strides=(1,1),padding='valid',activation='relu'))
#     model.add(tf.keras.layers.GroupNormalization(groups=3))
#     model.add(tf.keras.layers.AveragePooling2D(pool_size=(2,2),strides=None,padding='valid'))
#     model.add(tf.keras.layers.Conv2D(10,(2,2),strides=(1,1),padding='valid',activation='relu'))
#     model.add(tf.keras.layers.Flatten())
#     model.add(tf.keras.layers.Normalization())
#     model.add(tf.keras.layers.Dense(128,activation='relu'))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dropout(0.1)) #0.1?
#     model.add(tf.keras.layers.Dense(64,activation='relu'))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dropout(0.1)) #0.1?
#     model.add(tf.keras.layers.Dense(32,activation='relu'))
#     model.add(tf.keras.layers.Dense(10,activation='softmax'))
#     model.compile(optimizer="Adam",loss='categorical_crossentropy',metrics=['accuracy'])

In [70]:
with mirrored_strategy.scope():
    model=tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
    model.add(tf.keras.layers.Conv2D(30,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(30,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(30,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(20,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(20,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(20,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(15,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Conv2D(15,(3,3),strides=(1,1),padding='valid',activation='relu'))
#     model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(15,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.GroupNormalization(groups=3))
    model.add(tf.keras.layers.AveragePooling2D(pool_size=(2,2),strides=None,padding='valid')) #MaxPool??
    model.add(tf.keras.layers.Conv2D(10,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(10,(3,3),strides=(1,1),padding='valid',activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Normalization())
    model.add(tf.keras.layers.Dense(80,activation='selu'))
    model.add(tf.keras.layers.Dense(10,activation='softmax'))
    model.compile(optimizer="Adam",loss='categorical_crossentropy',metrics=['accuracy'])
    

In [71]:
# with mirrored_strategy.scope():
#     pool_net=tf.keras.Sequential()
#     pool_net.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
#     pool_net.add(tf.keras.layers.AveragePooling2D(pool_size=(2,2),strides=None,padding='same'))
#     pool_net.add(tf.keras.layers.BatchNormalization())
# #     pool_net.add(tf.keras.layers.AveragePooling2D(pool_size=(2,2),strides=None,padding='same'))
# #     pool_net.add(tf.keras.layers.BatchNormalization())
#     pool_net.add(tf.keras.layers.Flatten())
#     pool_net.add(tf.keras.layers.Normalization())
#     pool_net.add(tf.keras.layers.Dense(128,activation='relu'))
#     pool_net.add(tf.keras.layers.BatchNormalization())
#     pool_net.add(tf.keras.layers.Dropout(0.4))
#     pool_net.add(tf.keras.layers.Dense(64,activation='relu'))
#     pool_net.add(tf.keras.layers.Dense(10,activation='softmax'))
#     pool_net.compile(loss='categorical_crossentropy',metrics=['accuracy'])

In [72]:
# pool_net.summary()

In [73]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_68 (Conv2D)          (None, 30, 30, 30)        840       
                                                                 
 batch_normalization_57 (Bat  (None, 30, 30, 30)       120       
 chNormalization)                                                
                                                                 
 conv2d_69 (Conv2D)          (None, 28, 28, 30)        8130      
                                                                 
 batch_normalization_58 (Bat  (None, 28, 28, 30)       120       
 chNormalization)                                                
                                                                 
 conv2d_70 (Conv2D)          (None, 26, 26, 30)        8130      
                                                                 
 batch_normalization_59 (Bat  (None, 26, 26, 30)      

In [74]:
with mirrored_strategy.scope():
    checkpoint= tf.keras.callbacks.ModelCheckpoint(filepath='/kaggle/working/cifar_10_classifier_10.h5',
                                                   save_weights_only=False,monitor='val_accuracy',
                                                   save_best_only=True,save_freq="epoch",)
    early_stopping= tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,
                                                     restore_best_weights=True)
    def lr_scheduler(epoch,lr,epochs=200):
        initial=0.005
        if epoch<epochs*0.1:
            return initial
        elif epoch>epochs*0.1 and epoch<epochs*0.25:
            lr*=tf.math.exp(-0.1)
            return lr
        else:
            lr*=tf.math.exp(-0.008)
            return lr
    lr_scheduling=tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
    lr=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.2,patience=3,verbose=1,
                                            mode="min",min_delta=0.01,cooldown=0,min_lr=0)

In [75]:
# with mirrored_strategy.scope():
#     history = pool_net.fit(train_data.batch(100),epochs=200,validation_data=val_data.batch(100),
#                         callbacks=[checkpoint,early_stopping],use_multiprocessing=True)

In [76]:
#Implement Performance Scheduling?????????

In [ ]:
with mirrored_strategy.scope():
    history = model.fit(train_data.batch(8),epochs=200,validation_data=val_data.batch(8),
                        callbacks=[checkpoint,early_stopping,lr],use_multiprocessing=True)

Epoch 1/200
5625/5625 [==============================] - 94s 15ms/step - loss: 1.8631 - accuracy: 0.3132 - val_loss: 1.6158 - val_accuracy: 0.3976 - lr: 0.0010
Epoch 2/200
5625/5625 [==============================] - 85s 15ms/step - loss: 1.5405 - accuracy: 0.4424 - val_loss: 1.3703 - val_accuracy: 0.5062 - lr: 0.0010
Epoch 3/200
5625/5625 [==============================] - 85s 15ms/step - loss: 1.3617 - accuracy: 0.5108 - val_loss: 1.2240 - val_accuracy: 0.5652 - lr: 0.0010
Epoch 4/200
5625/5625 [==============================] - 85s 15ms/step - loss: 1.2488 - accuracy: 0.5557 - val_loss: 1.1680 - val_accuracy: 0.5818 - lr: 0.0010
Epoch 5/200
5625/5625 [==============================] - 85s 15ms/step - loss: 1.1737 - accuracy: 0.5810 - val_loss: 1.1360 - val_accuracy: 0.5968 - lr: 0.0010
Epoch 6/200
5625/5625 [==============================] - 85s 15ms/step - loss: 1.1206 - accuracy: 0.6006 - val_loss: 1.0661 - val_accuracy: 0.6232 - lr: 0.0010
Epoch 7/200
5625/5625 [=================

In [ ]:
model=tf.keras.models.load_model("/kaggle/working/cifar_10_classifier_10.h5")

In [ ]:
with mirrored_strategy.scope():
    predictions=model.predict(test_data.batch(10000))

In [ ]:
def outputs(x):
    a = np.zeros(x.shape)
    a[np.where(x==np.max(x))] = 1
    return a

In [ ]:
for i in range(len(predictions)):
     predictions[i]=outputs(predictions[i])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
y_test=np.concatenate([y for x, y in test_data.batch(10000)],axis=0)

In [ ]:
accuracy_score(predictions,y_test)